# Load Horizons Data into KeplerDB

In [104]:
import numpy as np
import pandas as pd
import re

In [3]:
with open("../data/jpl/horizons/planets/0010_sun.txt") as fh:
    lines = fh.readlines()

In [23]:
lines[29]

'Target body name: Sun (10)                        {source: DE431mx}\n'

In [6]:
lines[47]

'$$SOE\n'

In [8]:
lines[7489]

'$$EOE\n'

In [28]:
line = lines[29]
line[0:18]

'Target body name: '

In [105]:
def hrzn_target_body(lines):
    """Search horizons download file for name of target body"""
    for line in lines:
        if line[0:18] == 'Target body name: ':
            target_body = line[18:]
            break
    # regex search pattern
    p = re.compile('(^[\w]+) \(([\d]+)\)[\s]+\{source: ([\w]+)\}\n$')
    # extract name, number, and source from the target body string
    srch = p.search(target_body)
    body_name = srch.group(1)
    body_number = srch.group(2)
    integration_source = srch.group(3)
    return (body_name, body_number, integration_source)

In [106]:
hrzn_target_body(lines)

('Sun', '10', 'DE431mx')

In [32]:
tgt = hrzn_target_body(lines)

In [33]:
tgt

'Sun (10)                        {source: DE431mx}\n'

In [34]:
import re

In [99]:
p = re.compile('(^[\w]+) \(([\d]+)\)[\s]+\{source: ([\w]+)\}\n$')

In [100]:
p.match(tgt)

<re.Match object; span=(0, 50), match='Sun (10)                        {source: DE431mx}>

In [101]:
p.search(tgt).group(1)

'Sun'

In [102]:
p.search(tgt).group(2)

'10'

In [103]:
p.search(tgt).group(3)

'DE431mx'

In [11]:
def hrzn_find_start(lines):
    """Search horizons download file for first line of data"""
    for i, line in enumerate(lines):
        if line == '$$SOE\n':
            return i+1

In [15]:
def hrzn_find_end(lines):
    """Search horizons download file for last line of data"""
    n = len(lines)
    for i, line in enumerate(lines[::-1]):
        if line == '$$EOE\n':
            return n-i-1

In [18]:
i0 = hrzn_find_start(lines)

In [19]:
i1 = hrzn_find_end(lines)

In [20]:
lines[i0]

'2440400.500000000, A.D. 1969-Jun-28 00:00:00.0000,      39.699541,  4.502508784640554E-03,  8.096106239346330E-04, -6.102211252799528E-05, -3.517495360755232E-07,  5.637065280127068E-06, -1.438024411038287E-08,\n'

In [21]:
lines[i0:i1][-1]

'2477600.500000000, A.D. 2071-May-04 00:00:00.0000,      69.185457, -6.917795042183935E-04, -2.526351440007831E-03, -2.111311155652803E-06,  4.181688755397857E-06, -4.175320273026232E-06, -8.513458003532645E-08,\n'